# Tutorial: Extracting Information from Text Using OpenAI's API

This tutorial demonstrates how to use OpenAI's API to extract information from a text input. We will go through the steps of setting up the OpenAI client, making requests to the API, and processing the responses. The example involves summarizing a Danish news article and extracting structured information from it.

## Installation and Imports
First, we need to install the required library and import necessary modules.


In [ ]:
!pip install openai -q

In [ ]:
# Import required libraries
from openai import OpenAI
from google.colab import userdata
import json
from pydantic import BaseModel, Field

Setting Up the OpenAI Client

We will set up the OpenAI client using a custom API key and base URL.

In [ ]:
# Setup OpenAI client with custom API key and base URL
TOGETHER_API_KEY = userdata.get('TOGETHER_API_KEY')

### Summarizing Text

We will call the language model to summarize a given Danish text into a single sentence.

In [ ]:
# Create client
client = OpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=TOGETHER_API_KEY
)

In [ ]:
text = """
En nu 16-årig dreng fra Grenaa er blevet idømt otte års fængsel for et knivstikkeri i november sidste år på letbanestationen i Grenaa, hvor en 15-årig dreng mistede livet.
Et nævningeting ved Retten i Randers har kendt den 16-årige skyldig i drab efter at have stukket den 15-årige i brystkassen med en dolk.
Han er også kendt skyldig i forsøg på grov vold mod en anden dreng, der også var til stede.
De to unge havde i længere tid haft nogle uoverensstemmelser, og de mødtes en søndag aften sammen med andre unge på letbanestationen midt i Grenaa.
Her tog den 15-årige fat i den 16-årige og gav ham et knytnæveslag i ansigtet. Derefter tog den 16-årige sin dolk frem og stak ham.
Knivstikket ramte den 15-årige i hjertet, og han døde kort efter.
"""

In [ ]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    #model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    model="meta-llama/Llama-3-70b-chat-hf",

    messages=[
        {
            "role": "system",
            "content": "Du er en hjælpsom assistent, der udtrækker information fra dansk tekst. Svar på dansk",
        },
        {
            "role": "user",
            "content": "Opsumer følgende til 1 kort sætning: " + text ,
        },
    ],
)

output = chat_completion.choices[0].message.content

In [ ]:
print(output)

## Creating a User Object and extracting structured info
We will define a schema for a user and call the API to create a user object based on this schema.

In [ ]:
# Define the schema for the output.
class User(BaseModel):
    name: str = Field(description="user name")
    address: str = Field(description="address")

In [ ]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    response_format={"type": "json_object", "schema": User.model_json_schema()},
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that answers in JSON.",
        },
        {
            "role": "user",
            "content": "Create a user named Alice, who lives in 42, Wonderland Avenue.",
        },
    ],
)

created_user = json.loads(chat_completion.choices[0].message.content)
print(json.dumps(created_user, indent=2))

## Extracting Case Details
We will define a schema for case details and extract relevant information from the given text.

In [ ]:
class CaseDetails(BaseModel):
    incident_date: str = Field(description="Date of the incident")
    incident_location: str = Field(description="Location of the incident")
    incident_description: str = Field(description="Description of the incident")
    victim_age: int = Field(description="Age of the victim")
    victim_outcome: str = Field(description="Outcome for the victim")
    perpetrator_age: int = Field(description="Age of the perpetrator")
    perpetrator_sentence: str = Field(description="Sentence given to the perpetrator")
    perpetrator_charges: list[str] = Field(description="Charges against the perpetrator")
    trial_court: str = Field(description="Court where the trial was held")
    trial_verdict: str = Field(description="Verdict of the trial")

In [ ]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    response_format={"type": "json_object", "schema": CaseDetails.model_json_schema()},
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that answers in JSON.",
        },
        {
            "role": "user",
            "content": "Extract case informatin form following.: " + text,
        },
    ],
)

created_user = json.loads(chat_completion.choices[0].message.content)
print(json.dumps(created_user, indent=2))

In [ ]:
json_schema = """{
  "properties": {
    "incident_date": {
      "description": "Dato for hændelsen",
      "title": "Hændelsesdato",
      "type": "string"
    },
    "incident_location": {
      "description": "Sted for hændelsen",
      "title": "Hændelsessted",
      "type": "string"
    },
    "incident_description": {
      "description": "Beskrivelse af hændelsen",
      "title": "Hændelsesbeskrivelse",
      "type": "string"
    },
    "victim_age": {
      "description": "Offerets alder",
      "title": "Offerets alder",
      "type": "integer"
    },
    "victim_outcome": {
      "description": "Resultat for offeret",
      "title": "Offerets resultat",
      "type": "string"
    },
    "perpetrator_age": {
      "description": "Gerningsmandens alder",
      "title": "Gerningsmandens alder",
      "type": "integer"
    },
    "perpetrator_sentence": {
      "description": "Straf tildelt gerningsmanden",
      "title": "Gerningsmandens straf",
      "type": "string"
    },
    "perpetrator_charges": {
      "description": "Anklager mod gerningsmanden",
      "items": {
        "type": "string"
      },
      "title": "Gerningsmandens anklager",
      "type": "array"
    },
    "trial_court": {
      "description": "Retten hvor retssagen blev afholdt",
      "title": "Retssal",
      "type": "string"
    },
    "trial_verdict": {
      "description": "Dom i retssagen",
      "title": "Retssagens dom",
      "type": "string"
    }
  },
  "required": [
    "incident_date",
    "incident_location",
    "incident_description",
    "victim_age",
    "victim_outcome",
    "perpetrator_age",
    "perpetrator_sentence",
    "perpetrator_charges",
    "trial_court",
    "trial_verdict"
  ],
  "title": "Sagsdetaljer",
  "type": "object"
}"""

In [ ]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    model="meta-llama/Llama-3-70b-chat-hf",
    #response_format={"type": "json_object", "schema": CaseDetails.model_json_schema()},
    messages=[
        {
            "role": "system",
            "content": "Du er en hjælpsom assistent, der udtrækker information fra dansk tekst som JSON. Kun JSON. Ingen forklaringer. Ingen ekstra info!!!",
        },
        {
            "role": "user",
            "content": "udtræk fra følgende tekst.: " + text + "Brug følgende JSON schema: " + json_schema,
        },
    ],
)



In [ ]:
created_user = json.loads(chat_completion.choices[0].message.content)
print(json.dumps(created_user, ensure_ascii=False, indent=2))


In [ ]:
text_2 = """

En 22-årig mand fra Aalborg er blevet idømt tre års fængsel for narkotikahandel og besiddelse af ulovlige våben.

Et nævningeting ved Retten i Aalborg har kendt den 22-årige skyldig i at have solgt betydelige mængder kokain og ecstasy i Aalborg-området. Han er også kendt skyldig i besiddelse af en ulovlig pistol og en større mængde ammunition, som blev fundet under en ransagning af hans lejlighed.

Den 22-årige blev arresteret i december sidste år efter en længere efterforskning, hvor politiet havde overvåget hans aktiviteter. Retsagen afslørede, at han havde været involveret i narkotikahandel i mindst to år og havde etableret et stort netværk af kunder.

Under ransagningen fandt politiet også et større kontantbeløb, som menes at stamme fra salget af de ulovlige stoffer. Den 22-årige nægtede sig skyldig, men beviserne mod ham var overvældende, og han blev dømt på alle punkter.
"""

In [ ]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    model="meta-llama/Llama-3-8b-chat-hf",
    #response_format={"type": "json_object", "schema": CaseDetails.model_json_schema()},
    messages=[
        {
            "role": "system",
            "content": "Du er en hjælpsom assistent, der udtrækker information fra dansk tekst som JSON. Kun JSON. Ingen forklaringer. Ingen ekstra info!!!",
        },
        {
            "role": "user",
            "content": "udtræk fra følgende tekst.: " + text + "Brug følgende JSON schema: " + json_schema,
        },
                {
            "role": "assistant",
            "content": """{
  "properties": {
    "incident_date": "01-11-2023",
    "incident_location": "Letbanestationen i Grenaa",
    "incident_description": "Knivstikkeri",
    "victim_age": 15,
    "victim_outcome": "Mistede livet",
    "perpetrator_age": 16,
    "perpetrator_sentence": "Otte års fængsel",
    "perpetrator_charges": [
      "Drab",
      "Forsøg på grov vold"
    ],
    "trial_court": "Retten i Randers",
    "trial_verdict": "Skyldig"
  }
}""",
        },
                {
            "role": "user",
            "content": "udtræk fra følgende tekst.: " + text_2 + "Brug følgende JSON schema: " + json_schema,
        },
    ],
)



In [ ]:
chat_completion.choices[0].message.content

In [ ]:
created_user = json.loads(chat_completion.choices[0].message.content)
print(json.dumps(created_user, ensure_ascii=False, indent=2))
